# Getting started with langchain - question answering

This notebook walks through how to use LangChain for question answering over a given document. 

Large language models (LLMs) are emerging as a transformative technology, enabling developers to build applications that they previously could not. However, using these LLMs in isolation is often insufficient for creating a truly powerful app - the real power comes when you can combine them with other sources of computation or knowledge.

And that is what we will do here!

### Sources
- GitHub: https://github.com/hwchase17/langchain
- Documents: https://python.langchain.com/en/latest/use_cases/question_answering.html
- PyPi: https://pypi.org/project/langchain/

### Contents
0. Install packages
1. Prepare data
2. Search the document
2. Call the LLM to do generative AI

## 0. Install packages

In [1]:
!pip install langchain
!pip install openai

In [4]:
pip install langchain

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip show langchain

Name: langchain
Version: 0.0.339
Summary: Building applications with LLMs through composability
Home-page: https://github.com/langchain-ai/langchain
Author: 
Author-email: 
License: MIT
Location: /Users/michielbontenbal/anaconda3/lib/python3.10/site-packages
Requires: aiohttp, anyio, async-timeout, dataclasses-json, jsonpatch, langsmith, numpy, pydantic, PyYAML, requests, SQLAlchemy, tenacity
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install langchain --upgrade

  Using cached packaging-23.2-py3-none-any.whl.metadata (3.2 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.4/802.4 kB 4.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.7/228.7 kB 4.7 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 1.4 MB/s eta 0:00:00
Using cached packaging-23.2-py3-none-any.whl (53 kB)
  Attempting uninstall: packaging
    Found existing installation: packaging 23.1
    Uninstalling packaging-23.1:
      Successfully uninstalled packaging-23.1
  Attempting uninstall: langsmith
    Found existing installation: langsmith 0.0.66
    Uninstalling langsmith-0.0.66:
      Successfully uninstalled langsmith-0.0.66
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.339
    Uninstalling langchain-0.0.339:
      Successfully uninstalled langchain-0.0.339
ERROR: pip's dependency resolver does not c

In [ ]:
from langchain_community.llms import Ollama
llm= Ollama(model='llama2')
llm('Tell me about the history of AI')

/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


## 1. Settings

In [2]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.indexes.vectorstore import VectorstoreIndexCreator
import config

ImportError: cannot import name 'URL' from 'sqlalchemy' (/Users/michielbontenbal/anaconda3/lib/python3.10/site-packages/sqlalchemy/__init__.py)

In [ ]:
#store your openai api key as an environment variable - as required by OpenAI (see previous tutorial)
import os
import config # I've created a config.py file that stores my password
os.environ['OPENAI_API_KEY'] = config.openai_key #but don't keep it in your source code :-)

## 2. Get the data and create embeddings

In [ ]:
#we will use the wget library to download the file 
import wget
filename = wget.download('https://github.com/hwchase17/chat-your-data/blob/master/state_of_the_union.txt')
filename

In [ ]:
#split the document into chunks
with open("state_of_the_union.txt") as f:
    state_of_the_union = f.read()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_text(state_of_the_union)

#create the embeddings
embeddings = OpenAIEmbeddings()

## 2. Search the document
A first step can be to search your document. This way we will find the location in the document where words of the query are mentioned. The result of this search is the relevant part of the document
(In the next step we will use this result and feed it to the LLM so it can generate a nice response.)

In [ ]:
#first we will create docsearch variable. 
docsearch = Chroma.from_texts(texts, embeddings, metadatas=[{"source": str(i)} for i in range(len(texts))]).as_retriever()

In [ ]:
#first tr
query = "What did the president say about Justice Breyer"
docs = docsearch.get_relevant_documents(query)

In [ ]:
#print the best result
docs[0]

## 3. Call the LLM to generate an answer
If you just want to get started as quickly as possible, this is the fasted way to do it.

In [ ]:
#import two more liberies
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [ ]:
#create the chain, give the pro
chain = load_qa_chain(OpenAI(temperature=0), chain_type="stuff")
#create your prompt
query = "What did the president say about Justice Breyer"
chain.run(input_documents=docs, question=query)

## Conclusion
That's it! We've taken a text document from the web and did document search with it. Also we used a Large Language Model to generate a good answer for it!